# Création d'un Master Bias à partir de fichiers FITS

Ce notebook combine plusieurs images de bias pour créer un **master bias** par médiane.

Étapes :
1. Charger les fichiers `.fits` de bias.
2. Vérifier leur taille et cohérence.
3. Empiler les images.
4. Calculer la médiane pixel par pixel.
5. Sauvegarder le résultat sous `master_bias.fits`.
6. Afficher une visualisation et des statistiques.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pathlib import Path

# Dossier contenant les fichiers bias
bias_dir = Path('/Volumes/Astro/ohp/lyon/Bias')  # à adapter

# Liste de tous les fichiers FITS
bias_files = sorted(bias_dir.glob('M_32*.fits'))
print(bias_files)
print(f"Nombre de fichiers trouvés : {len(bias_files)}")
if len(bias_files) == 0:
    raise FileNotFoundError("Aucun fichier .fits trouvé dans le dossier spécifié.")

In [ ]:
# Lecture de toutes les images
bias_data = []
for file in bias_files:
    print(f"filename : {file}")
    with open(file, "rb") as f:
        head = f.read(2880)  # lis les 2880 premiers octets (1 bloc FITS)
        print(head[:100])
    with fits.open(file) as hdul:
        data = hdul[0].data.astype(np.float32)
        bias_data.append(data)

bias_data = np.array(bias_data)
print(f"Dimensions des images : {bias_data.shape}")

In [ ]:
# Calcul du master bias (médiane pixel par pixel)
master_bias = np.median(bias_data, axis=0)
print(f"Master bias calculé : dimensions {master_bias.shape}")

In [ ]:
# Sauvegarde du master bias dans un fichier FITS
output_path = Path('master_bias.fits')
fits.writeto(output_path, master_bias.astype(np.float32), overwrite=True)
print(f"Master bias sauvegardé dans {output_path.resolve()}")

In [ ]:
# Affichage du master bias
plt.figure(figsize=(8, 6))
plt.imshow(master_bias, cmap='gray', origin='lower', vmin=np.percentile(master_bias, 5), vmax=np.percentile(master_bias, 95))
plt.colorbar(label='ADU')
plt.title('Master Bias')
plt.show()

print(f"Moyenne : {np.mean(master_bias):.2f} ADU")
print(f"Écart-type : {np.std(master_bias):.2f} ADU")